# 📘 Aula: Construindo e Convertendo Funçõpes em Ferramentas com LLMs

### 1 - 🧱 Importações e Setup Inicial

In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq


📌  Aqui você está usando a biblioteca llama_index, que serve como uma camada de orquestração e integração entre modelos de linguagem e ferramentas.

> - Settings: configurações globais do LlamaIndex.

> - Groq: driver para utilizar os modelos hospedados na Groq API (foco em alto desempenho). 

---

### 🔐  2. Configuração do modelo Groq

In [ ]:
import os
llm = Groq(model="llama-3.3-70b-versatile",
           api_key=os.environ["GROQ_API_KEY"],)

📌 Aqui você:

> - Importa o módulo os para acessar variáveis de ambiente.

> - Cria uma instância do LLM usando o modelo LLaMA 3.3 70B via Groq.

> - Usa a variável GROQ_API_KEY que deve estar configurada no seu .env ou ambiente de execução.

---

### 🧮 3. Criar uma função de calculo de imposto de renda, para se transformar em uma 🛠️Tool (🛠️ferramenta)

In [ ]:
def calcular_imposto_renda(rendimento: float) -> str:
    """
    Calcula o imposto de renda com base no rendimento anual.
    Args:
        rendimento (float): Rendimento anualdo individuo.
    Returns:
        str: Valor do imposto de renda a ser pago.
    """
    if rendimento <= 2000:
        return "Você está isento de pagar imposto de renda."
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"Você deve pagar R$ {imposto:.2f} baseado em um rendimento de R$ {rendimento:.2f}."
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + (3000 * 0.10)
        return f"Você deve pagar R$ {imposto:.2f} baseado em um rendimento de R$ {rendimento:.2f}."
    else:
        imposto = (rendimento - 10000) * 0.20 + (5000 * 0.15) + (3000 * 0.10)
        return f"Você deve pagar R$ {imposto:.2f} baseado em um rendimento de R$ {rendimento:.2f}."

📌 Objetivo: criar uma função que calcule o imposto de renda com base em faixas de rendimento anual.

> - Até R$ 2.000: isento

> - R$ 2.001 a R$ 5.000: 10% sobre o que exceder R$ 2.000

> - R$ 5.001 a R$ 10.000: 15% sobre o que exceder R$ 5.000 + 10% da faixa anterior

> - Acima de R$ 10.000: 20% sobre o que exceder R$ 10.000 + 15% da faixa anterior + 10% da primeira faixa

🎯 Essa função será usada depois como uma ferramenta (Tool) acessível a LLMs.

---

### 🔄 4. Convertendo Função em Ferramenta (Tool)

In [ ]:
from llama_index.core.tools import FunctionTool

#### 🧪 Definindo Função

In [ ]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular imposto de renda",
    description=(
        "Calcula o imposto de renda com base no rendimento anual do individuo."
        "Argumento: rendimento (float)."
        "Retorna o valor do imposto devido de acordo com as faixas de endimento."
    )
)


📌 Aqui você transforma a função calcular_imposto_renda em uma ferramenta utilizável pelo agente.

---

# ⚠️ Usando o Agente

### 🧠 5. Criando um Agente que usa Ferramenta

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)


📌 Criação de um Agente Worker que pode utilizar a ferramenta ferramenta_imposto_renda. Permite chamadas paralelas e faz uso do modelo configurado.

---

### 📦 6.Executar Agente 

In [ ]:
from llama_index.core.agent import AgentRunner

#### 🏷️ Importar Agent runner

In [ ]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [ ]:
response = agent_imposto.chat("""
    QUal o imposto a ser pago por um rendimento anual de R$ 5000?
    """
)

📌 Aqui o AgentRunner está executando o agente com base em um prompt. O agente analisa a pergunta e usa a função-ferramenta para responder.

---

# 📡 AGENTES MULTITAREFAS

### 📚 7.  Nova Ferramenta: Buscar Artigos no ArXiv

##### 📂 Importar ARXIV

In [ ]:
import arxiv

def consulta_artigos(titulo: str) -> str:
    """
    Consulta os artigos na base de dados ArXiv e retorna resultados formatados.
    """
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    resultados = [
    f"Título: {artigo.title}\n"
    f"Categoria: {artigo.primary_category}\n"
    f"Link: {artigo.entry_id}\n"
    for artigo in busca.results()
    ]
    
    return "\n\n".join(resultados)

📌 Você define uma função que busca artigos científicos no ArXiv com base no título.

---

#### 🛠️ 8. Transformar busca em ferramenta

In [ ]:
consulta_artigos_tool = FunctionTool.from_defaults(
    fn=consulta_artigos,
    name="Consulta artigos ArXiv",
    description=(
        "Consulta artigos na base de dados ArXiv com base no título fornecido."
        "Argumento: titulo (str)."
        "Retorna uma lista de artigos relevantes com título, categoria e link."
    )
)

📌 Agora o LLM poderá usar essa ferramenta para fazer buscas reais de pesquisa científica.

---

## 🧩 Agente Worker - Agente com Múltiplas ferramentas 🛠️🧰

> - usar o
```python
llm=llm
```
> - para chamar o llm do codigo, pois ele usa como padrão o OpenAI

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

📌 O agente agora pode escolher entre calcular imposto ou buscar artigos científicos, dependendo do seu input.

---

#### ⚙️ Executar o agent_runner

In [ ]:
agent = AgentRunner(agent_worker)
response = agent.chat("""
 Me retorne artigos sobre LangChain na educação
    """
)

# 🌐 Integração com Tavily

### 🔐 Comfirmação de uso da OS

In [ ]:
import os
from dotenv import load_dotenv,find_dotenv
_= load_dotenv(find_dotenv())

#### ⚙️ importando a API KEY

In [ ]:
tavily_key = os.environ.get("TAVILY_API_KEY")

#### 🔗 Intagrando API key do tavily com Llama_Index

In [ ]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

📌 Conecta a API Tavily (plataforma de busca científica) ao LlamaIndex.

---

#### 🛠️ Taviily como Ferramenta (Função de Busca)

In [ ]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)


#### 🔎 Usando Apenas Tavily para fazer pesquisa

In [ ]:
tavily_tool.search("Me retorne artigos cientificos sobre LangChain", max_results=3)

# 🧩 Incorporar o tavily como ferramenta para o agente

#### importar 

In [ ]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description=(
        "Busca artigos científicos e informações na base de dados Tavily."
    )
)


#### 🛠️ Criar agent worker

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("""
    Me retorne artigos sobre LLM e LangChain
    """
)

In [ ]:
print(response)

## 📝 Usar arquivos PDF


In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

#### 🧠 Fazer "upload" dos PDFs para o sistema

In [ ]:
url = "files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()


In [ ]:
url = "files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

#### 🚨 Gerar os Embeddings com HuggingFace

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#### ⚠️ Instalando Hugging Face

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-large"
)

#### 🗂️ .Gerando Índices e Persistência

In [ ]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

📌 Cria índices vetoriais para os PDFs, permitindo buscas inteligentes dentro dos textos.


#### 🔎 .Criando Query Engines

In [ ]:
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

📌 Constrói mecanismos de busca para os documentos usando LLM + embeddings.

### ⚙️ Engine de Busca

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

In [ ]:
storage_context = StorageContext.from_defaults(
    persist_dir="artigo"
)
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
    persist_dir="tutorial"
)
tutorial_index = load_index_from_storage(storage_context)

#### teste de engine

In [ ]:
artigo_engine = artigo_index.as_query_engine(
    similarity_top_k=3, llm=llm
)

tutorial_engine = tutorial_index.as_query_engine(
    similarity_top_k=3, llm=llm
)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

#### 🧰 . Incorporando como ferramentas do agente

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre LLM e LangChain."
                    "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            )
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "Fornece informações sobre casos de uso e aplicações em LLMs."
                    "Use uma pergunta detalhada em texto simples como entrada para a ferramenta."
            )
        )
    )
]

📌 Agora seus PDFs se tornam ferramentas que o LLM pode consultar, como se fossem especialistas.

---

## 🤖 20. Agente final: pesquisa nos documentos

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

📌 Esse agente consegue responder perguntas específicas com base no conteúdo dos seus PDFs usando o response!

In [ ]:
response = agent_document.chat("""
    Quais as principais aplicações posso construir com LLM e LangChain?
    """
)

In [ ]:
response = agent_document.chat("""
    Quais as principais tendencias em langchain e llm ?
    """
)

# 🤖 Agente ReAct

#### 📂 Importando agente ReAct

In [ ]:
from llama_index.core.agent import ReActAgent

#### 🔧 Configurando agente

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

#### 💬 RESPOSTA

In [ ]:
response = agent.chat("""  
    Quais as principais ferramentas usadas em LangChain?
    """
)

In [ ]:
response = agent.chat("""
Quais as principaios tendencias em langchain que eu deveria estudar?
    """
)